# Hive Assignment

> CA675

Author : Shubham Rai
Student Number : 21261161

# Data Extraction

Data set extracted from following link 
https://data.stackexchange.com/stackoverflow/query/new

### Query used to extract data
As data extraction limit is 50000 we need to extract 4 files with help of row number.


```
SELECT 
*
FROM 
(
SELECT 
ROW_NUMBER() OVER (ORDER BY VIEWCOUNT DESC) AS RN, 
Id,
PostTypeId,
AcceptedAnswerId,
ParentId,
CreationDate,
DeletionDate,
Score,
ViewCount,
OwnerUserId,
OwnerDisplayName,
LastEditorUserId,
LastEditorDisplayName,
LastEditDate,
LastActivityDate,
Title,
Tags,
AnswerCount,
CommentCount,
FavoriteCount,
ClosedDate,
CommunityOwnedDate,
ContentLicense
FROM POSTS WHERE VIEWCOUNT IS NOT NULL
) AS T
WHERE RN BETWEEN 150000 AND 200000
```



# Starting Hive Server

bin/hiveserver2
bin/beeline -n hdoop -u jdbc:hive2://localhost:10000

# Create Table in HIVE
As delimited file formated has issues while reading quoted characters we are using SERDE format to read CSV and insert data in HIVE


In [ ]:
create table stackexchange(RN int, Id int, PostTypeId int,AcceptedAnswerId int,ParentId int,CreationDate timestamp,DeletionDate timestamp,Score BIGINT,ViewCount BIGINT,OwnerUserId varchar(255),OwnerDisplayName varchar(255),LastEditorUserId varchar(255),LastEditorDisplayName varchar(255),LastEditDate timestamp,LastActivityDate timestamp,Title timestamp,Tags varchar(255),AnswerCount int,CommentCount int,FavoriteCount int,ClosedDate timestamp,CommunityOwnedDate timestamp,ContentLicense varchar(255))  ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde';

# Put Data on HDFS

hadoop fs -put < local_path  to < hdfs_location > 

# Task 2 & 3

### 2.2.1) Querying top 10 posts by score

In [4]:
from pyhive import hive
from tabulate import tabulate
import pandas as pd


host_name = "localhost"
port = 10000
user = "hdoop"
password = "a"
database="default"

def hiveconnection(host_name, port, user,password, database):
    conn = hive.Connection(host=host_name, port=port, username=user, password=password,
                           database=database, auth='CUSTOM')
    return conn


conn = hiveconnection(host_name, port, user,password, database)
cur = conn.cursor()

## Usage example from https://github.com/dropbox/PyHive


In [80]:
cur.execute('select ID, Title, Score from stackexchange_view order by score desc limit 10')
result = cur.fetchall()
print(tabulate(result, tablefmt='orgtbl'))

| 11227809 | Why is processing a sorted array faster than processing an unsorted array? | 25893 |
|   927358 | How do I undo the most recent local commits in Git?                        | 23274 |
|  2003505 | How do I delete a Git branch locally and remotely?                         | 18451 |
|   292357 | What is the difference between 'git pull' and 'git fetch'?                 | 12796 |
|   231767 | What does the "yield" keyword do?                                          | 11512 |
|   477816 | What is the correct JSON content type?                                     | 10894 |
|   348170 | How do I undo 'git add' before commit?                                     | 10045 |
|  5767325 | How can I remove a specific item from an array?                            |  9877 |
|  6591213 | How do I rename a local Git branch?                                        |  9747 |
|  1642028 | What is the "-->" operator in C/C++?                                       |  9539 |


### 2.2.2) The top 10 users by post score 

In [8]:
### Join data as usernames data was extracted and added post data pulling

cur.execute("""
SELECT DISTINCT u.displayname,stackexchange.score,stackexchange.owneruserid FROM
(
    select
           OwnerUserId,
           sum(score) as score
    from stackexchange_view
    group by OwnerUserId
    order by score desc
    LIMIT 11
)stackexchange
LEFT JOIN usernames u
ON stackexchange.owneruserid = u.id
ORDER BY stackexchange.score DESC
""")
result = cur.fetchall()
print(tabulate(result, tablefmt='orgtbl'))

|                 | 448906 |        |
| GManNickG       |  37606 | 87234  |
| readonly        |  28739 | 4883   |
| e-satis         |  26728 | 9951   |
| pupeno          |  25860 | 6068   |
| Hamza Yerlikaya |  23949 | 89904  |
| Joan Venge      |  23632 | 51816  |
| Ali             |  20156 | 49153  |
| TIMEX           |  19454 | 179736 |
| Matthew Rankin  |  19413 | 95592  |
| flybywire       |  19295 | 63051  |



### 2.2.3) The number of distinct users, who used the word “cloud” in one of their posts

## -- Note : Using Title and Tags field for counting cloud as we are receiving timeout when trying to pull body field from source

In [15]:
cur.execute("""
SELECT
     COUNT(DISTINCT owneruserid) as user_count
FROM stackexchange_view
WHERE title LIKE '%cloud%' or tags LIKE '%cloud%'
""")
result = cur.fetchall()
print(tabulate(result, tablefmt='orgtbl'))

| 357 |


In [77]:
### Get data for top 10 users from above list and pull data

df = pd.read_sql("""
SELECT
       owneruserid,
       title
from stackexchange_view
WHERE owneruserid
IN
      (
             87234,
             4883,
             9951,
             6068,
             89904,
             51816,
             49153,
             179736,
             95592,
             63051
          )
order by owneruserid""", conn)
top_10_user_id = list(df["owneruserid"].unique())


In [62]:
## References taken and custom function created by me. 
## Reference : https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a

from sklearn.feature_extraction.text import TfidfVectorizer


# Calculate sum() of TF-IDF and get top 10 words with highest TF-IDF and select only those columns
def calculate_tf_idf(df):
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True) # Remove Stop Words
    response = vectorizer.fit_transform(df["title"]) # Use title field for TF/IDF
    df_tfidf_sklearn = pd.DataFrame(response.toarray(),columns=vectorizer.get_feature_names_out())
    total_tf_idf = df_tfidf_sklearn.sum(axis = 0) # Remove sum of TF/IDF for getting top 10 most used words
    top_10_list = total_tf_idf.nlargest(10) # Get top 10 words per user
    top_10_words = list(top_10_list.index) # Get list of top 10 words
    df_tfidf_sklearn[top_10_words] # Select only top 10 words as column
    return df_tfidf_sklearn[top_10_words]


## For each of top 10 users and their top 10 words plot TF/IDF table for their correspnding comments

In [81]:
## For all 10 users create TF-IDF table for each user for their respective comments. 

for each_user in top_10_user_id: ## Iterate over all top 10 users
    username_id = str(each_user)
    filtered_data = df[(df['owneruserid']==username_id)] # Filter data only for selected user
    tf_idf_df = calculate_tf_idf(filtered_data) # pass on to above function
    print("For Username ID TF/IDF table : "+username_id) 
    tf_idf_df.insert(0, 'usernameid', username_id)# attach username ID field to dataframe
    display(tf_idf_df)
    print("******************************************************************************\n\n\n")
    

    


For Username ID TF/IDF table : 179736


,usernameid,python,django,javascript,js,node,use,string,dictionary,query,mysql
0,179736,0.338128,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,179736,0.000000,0.000000,0.0,0.275686,0.275686,0.0,0.000000,0.0,0.0,0.000000
2,179736,0.000000,0.000000,0.0,0.350733,0.350733,0.0,0.000000,0.0,0.0,0.000000
3,179736,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,179736,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.394849,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
109,179736,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
110,179736,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
111,179736,0.000000,0.288203,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
112,179736,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.398258


******************************************************************************



For Username ID TF/IDF table : 4883


,usernameid,python,git,ruby,table,process,list,rename,rails,difference,write
0,4883,0.000000,0.000000,0.000000,0.332494,0.000000,0.000000,0.362955,0.362955,0.000000,0.362955
1,4883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,4883,0.000000,0.000000,0.000000,0.000000,0.000000,0.416967,0.000000,0.000000,0.000000,0.000000
3,4883,0.000000,0.000000,0.427544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,4883,0.000000,0.000000,0.000000,0.000000,0.350231,0.000000,0.000000,0.000000,0.000000,0.000000
6,4883,0.000000,0.000000,0.521156,0.000000,0.000000,0.568902,0.000000,0.000000,0.000000,0.000000
7,4883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,4883,0.000000,0.402233,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,4883,0.000000,0.000000,0.000000,0.521156,0.000000,0.000000,0.568902,0.000000,0.000000,0.000000


******************************************************************************



For Username ID TF/IDF table : 49153


...

******************************************************************************



For Username ID TF/IDF table : 95592


,usernameid,install,pip,installed,python,virtualenv,cache,local,determining,flask,version
0,95592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,95592,0.000000,0.000000,0.360493,0.000000,0.360493,0.000000,0.000000,0.000000,0.000000,0.000000
2,95592,0.000000,0.000000,0.435556,0.000000,0.000000,0.000000,0.000000,0.519708,0.519708,0.519708
3,95592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,95592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,95592,0.347708,0.347708,0.000000,0.347708,0.347708,0.000000,0.000000,0.000000,0.000000,0.000000
6,95592,0.454195,0.454195,0.000000,0.000000,0.000000,0.541948,0.541948,0.000000,0.000000,0.000000
7,95592,0.000000,0.000000,0.000000,0.386479,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


******************************************************************************



For Username ID TF/IDF table : 9951


,usernameid,git,javascript,python,string,file,callable,android,dictionary,url,way
0,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
1,9951,0.000000,0.000000,0.000000,0.455000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
2,9951,0.541841,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
3,9951,0.000000,0.493950,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
4,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
5,9951,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.00000,0.000000,0.000000,0.000000
6,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.530479,0.000000,0.000000
7,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.468624,0.000000
8,9951,0.360453,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000
9,9951,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000


******************************************************************************



